Anova_glove_150features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Glove/glove_150_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,-0.067207,-0.387365,-0.683783,-0.489021,-0.016981,-0.153316,-2.143291,1.291261,0.097681,...,1.842933,1.380391,0.621629,-1.158252,0.798816,2.456636,-0.611517,2.123664,0.492321,-0.905096
1,1,-1.044469,-0.904845,0.362159,-0.018400,-1.041424,-0.356175,0.009922,-0.766948,0.717891,...,-0.531519,0.644047,0.231670,0.790292,0.314078,0.888807,0.135119,1.725640,-0.126132,0.430165
2,2,0.933923,-0.894353,0.477602,-0.549996,-0.102634,-0.060297,0.204495,-2.007398,0.385843,...,1.079009,1.411851,0.610612,-0.510759,-0.376822,0.969446,0.436492,0.798300,0.051008,-1.701203
3,3,-1.698108,0.746379,-1.876408,0.838482,0.834835,-0.797099,0.285148,1.227330,-0.550492,...,-0.026690,-1.126369,-0.671165,2.548123,1.119206,-0.204359,-0.041964,-0.571071,-1.604817,0.157689
4,4,0.520871,-2.014714,-0.798006,0.003402,-0.025342,-1.809653,-0.189463,-0.092498,0.554427,...,0.727781,0.397521,1.705182,-0.988835,-0.490491,0.779373,-1.263211,-0.449102,0.502192,-0.482567


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.067207,-0.387365,-0.683783,-0.489021,-0.016981,-0.153316,-2.143291,1.291261,0.097681,1.261490,...,1.842933,1.380391,0.621629,-1.158252,0.798816,2.456636,-0.611517,2.123664,0.492321,-0.905096
1,-1.044469,-0.904845,0.362159,-0.018400,-1.041424,-0.356175,0.009922,-0.766948,0.717891,-0.698628,...,-0.531519,0.644047,0.231670,0.790292,0.314078,0.888807,0.135119,1.725640,-0.126132,0.430165
2,0.933923,-0.894353,0.477602,-0.549996,-0.102634,-0.060297,0.204495,-2.007398,0.385843,-1.548081,...,1.079009,1.411851,0.610612,-0.510759,-0.376822,0.969446,0.436492,0.798300,0.051008,-1.701203
3,-1.698108,0.746379,-1.876408,0.838482,0.834835,-0.797099,0.285148,1.227330,-0.550492,2.038569,...,-0.026690,-1.126369,-0.671165,2.548123,1.119206,-0.204359,-0.041964,-0.571071,-1.604817,0.157689
4,0.520871,-2.014714,-0.798006,0.003402,-0.025342,-1.809653,-0.189463,-0.092498,0.554427,-0.393898,...,0.727781,0.397521,1.705182,-0.988835,-0.490491,0.779373,-1.263211,-0.449102,0.502192,-0.482567


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.38160723346069597


**Decision Tree**

In [17]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [18]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [19]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [20]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [21]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [22]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.546721,0.546721,0.592556,0.548785,0.30916,0.30916,0.34646,0.313035


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9532    0.9584    0.9558       361
           2     0.9796    0.9566    0.9680       553
           3     0.9432    0.9771    0.9599       306

    accuracy                         0.9623      1220
   macro avg     0.9587    0.9641    0.9612      1220
weighted avg     0.9627    0.9623    0.9623      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2830    0.0955    0.1429       157
           2     0.4630    0.8439    0.5979       237
           3     0.2564    0.0769    0.1183       130

    accuracy                         0.4294       524
   macro avg     0.3341    0.3388    0.2864       524
weighted avg     0.3578    0.4294    0.3426       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.546721,0.546721,0.592556,0.548785,0.309160,0.309160,0.346460,0.313035
1,RandomForest,0.962295,0.962295,0.962667,0.962342,0.429389,0.429389,0.357804,0.342590


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5217    0.3657    0.4300       361
           2     0.5669    0.7360    0.6404       553
           3     0.5944    0.4837    0.5333       306

    accuracy                         0.5631      1220
   macro avg     0.5610    0.5284    0.5346      1220
weighted avg     0.5604    0.5631    0.5513      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3145    0.2484    0.2776       157
           2     0.4731    0.5570    0.5116       237
           3     0.2149    0.2000    0.2072       130

    accuracy                         0.3760       524
   macro avg     0.3342    0.3351    0.3321       524
weighted avg     0.3615    0.3760    0.3660       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.552459,0.552459,0.629821,0.512631,0.410305,0.410305,0.385768,0.366827
1,RandomForest,0.962295,0.962295,0.962309,0.962288,0.437023,0.437023,0.383733,0.367269
2,Adaboost,0.563115,0.563115,0.560407,0.551297,0.375954,0.375954,0.361530,0.365970


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9662    0.8698    0.9155       361
           2     0.8883    0.9783    0.9312       553
           3     0.9441    0.8824    0.9122       306

    accuracy                         0.9221      1220
   macro avg     0.9329    0.9102    0.9196      1220
weighted avg     0.9253    0.9221    0.9217      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3402    0.2102    0.2598       157
           2     0.4746    0.7089    0.5685       237
           3     0.2740    0.1538    0.1970       130

    accuracy                         0.4218       524
   macro avg     0.3629    0.3576    0.3418       524
weighted avg     0.3845    0.4218    0.3839       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.552459,0.552459,0.629821,0.512631,0.410305,0.410305,0.385768,0.366827
1,RandomForest,0.962295,0.962295,0.962309,0.962288,0.437023,0.437023,0.383733,0.367269
2,Adaboost,0.563115,0.563115,0.560407,0.551297,0.375954,0.375954,0.361530,0.365970
3,GBM,0.922131,0.922131,0.925341,0.921744,0.421756,0.421756,0.384548,0.383878


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4805    0.5817    0.5263       361
           2     0.6979    0.4846    0.5720       553
           3     0.4586    0.5980    0.5191       306

    accuracy                         0.5418      1220
   macro avg     0.5457    0.5548    0.5392      1220
weighted avg     0.5736    0.5418    0.5452      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2865    0.3376    0.3099       157
           2     0.5063    0.3376    0.4051       237
           3     0.2597    0.3615    0.3023       130

    accuracy                         0.3435       524
   macro avg     0.3508    0.3456    0.3391       524
weighted avg     0.3793    0.3435    0.3511       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.546721,0.546721,0.592556,0.548785,0.309160,0.309160,0.346460,0.313035
1,RandomForest,0.962295,0.962295,0.962667,0.962342,0.429389,0.429389,0.357804,0.342590
2,SVM,0.541803,0.541803,0.573584,0.545243,0.343511,0.343511,0.379266,0.351056


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4987    0.5291    0.5134       361
           2     0.5900    0.7052    0.6425       553
           3     0.4716    0.2712    0.3444       306

    accuracy                         0.5443      1220
   macro avg     0.5201    0.5019    0.5001      1220
weighted avg     0.5333    0.5443    0.5295      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2761    0.2866    0.2812       157
           2     0.4825    0.5823    0.5277       237
           3     0.2667    0.1538    0.1951       130

    accuracy                         0.3874       524
   macro avg     0.3418    0.3409    0.3347       524
weighted avg     0.3671    0.3874    0.3714       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.552459,0.552459,0.629821,0.512631,0.410305,0.410305,0.385768,0.366827
1,RandomForest,0.962295,0.962295,0.962309,0.962288,0.437023,0.437023,0.383733,0.367269
2,Adaboost,0.563115,0.563115,0.560407,0.551297,0.375954,0.375954,0.361530,0.365970
3,GBM,0.922131,0.922131,0.925341,0.921744,0.421756,0.421756,0.384548,0.383878
4,SVM,0.559836,0.559836,0.551788,0.546700,0.339695,0.339695,0.317094,0.325406
5,KNN,0.544262,0.544262,0.533290,0.529543,0.387405,0.387405,0.367112,0.371360


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4424    0.3296    0.3778       361
           2     0.5664    0.4702    0.5138       553
           3     0.3252    0.5229    0.4010       306

    accuracy                         0.4418      1220
   macro avg     0.4447    0.4409    0.4309      1220
weighted avg     0.4692    0.4418    0.4453      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3248    0.2420    0.2774       157
           2     0.4900    0.4135    0.4485       237
           3     0.2754    0.4385    0.3383       130

    accuracy                         0.3683       524
   macro avg     0.3634    0.3647    0.3547       524
weighted avg     0.3872    0.3683    0.3699       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.552459,0.552459,0.629821,0.512631,0.410305,0.410305,0.385768,0.366827
1,RandomForest,0.962295,0.962295,0.962309,0.962288,0.437023,0.437023,0.383733,0.367269
2,Adaboost,0.563115,0.563115,0.560407,0.551297,0.375954,0.375954,0.361530,0.365970
3,GBM,0.922131,0.922131,0.925341,0.921744,0.421756,0.421756,0.384548,0.383878
4,SVM,0.559836,0.559836,0.551788,0.546700,0.339695,0.339695,0.317094,0.325406
5,KNN,0.544262,0.544262,0.533290,0.529543,0.387405,0.387405,0.367112,0.371360
6,GNB,0.441803,0.441803,0.469227,0.445274,0.368321,0.368321,0.387249,0.369888
